In [1]:
# 4.2.6 네거티브 샘플링의 샘플링 기법

import numpy as np

np.random.choice(10)

7

In [2]:
np.random.choice(10)

3

In [3]:
words = ['you','say','goodbye','I','hello','.']
np.random.choice(words)

'say'

In [4]:
np.random.choice(words, size=5)

array(['you', 'hello', '.', 'say', '.'], dtype='<U7')

In [5]:
np.random.choice(words, size=5, replace=False)

array(['I', 'hello', 'goodbye', 'you', '.'], dtype='<U7')

In [6]:
p=[0.5,0.1,0.05,0.2,0.05,0.1]
np.random.choice(words,p=p)

'.'

In [8]:
p=[0.7,0.29,0.01]
new_p=np.power(p,0.75)
new_p/=np.sum(new_p)
print(new_p)

[0.64196878 0.33150408 0.02652714]


In [9]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

In [10]:
corpus=np.array([0,1,2,3,4,1,2,3])
power=0.75
sample_size=2

sampler=UnigramSampler(corpus,power,sample_size)
target=np.array([1,3,0])
negative_sample=sampler.get_negative_sample(target)
print(negative_sample)

[[4 0]
 [1 2]
 [1 4]]
